In [1]:
!git clone https://github.com/ylacombe/musicgen-dreamboothing.git
!pip install -U git+https://github.com/huggingface/transformers

Cloning into 'musicgen-dreamboothing'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 152 (delta 14), reused 13 (delta 13), pack-reused 135 (from 1)
Receiving objects: 100% (152/152), 5.72 MiB | 33.07 MiB/s, done.
Resolving deltas: 100% (75/75), done.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-u2z47va_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-u2z47va_
  Resolved https://github.com/huggingface/transformers to commit 51d732709e5ae424e8fb6c4e58b72057a3e413c2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11363407 sha256=9a10eae984e23b93fe5a86c0cb69053d0fdcf74ad61bfba4af8ba1c3e1efe36a
  Stored in directory:

In [2]:
%pip install transformers peft torch accelerate sentencepiece "datasets[audio]>=2.12.0" wandb evaluate torchaudio soundfile "black~=23.1" "isort>=5.5.4" "ruff>=0.0.241,<=0.0.259" msclap librosa demucs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.

In [3]:
# Download audio data
!wget https://zenodo.org/records/10278066/files/edm_tr9_drm_id_001.tar.gz?download=1 -O edm_data.tar.gz

!mkdir edm_data

!tar xzf edm_data.tar.gz -C edm_data

--2025-05-31 17:53:54--  https://zenodo.org/records/10278066/files/edm_tr9_drm_id_001.tar.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4810529632 (4.5G) [application/octet-stream]
Saving to: ‘edm_data.tar.gz’

edm_data.tar.gz     100%[===================>]   4.48G  16.4MB/s    in 7m 45s  

2025-05-31 18:01:39 (9.87 MB/s) - ‘edm_data.tar.gz’ saved [4810529632/4810529632]



In [4]:
# !rm -rf *.wav

In [ ]:
import os
import csv

from datasets import DatasetDict

from transformers import AutoProcessor, AutoModelForTextToWaveform
import torch
import soundfile as sf

import IPython.display as ipd

2025-05-31 18:03:01.539356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748714581.740533      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748714581.801407      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True

In [7]:
def generate_caption(filename):
    file_ids = filename.split("_")
    # print(file_ids)
    bpm = file_ids[0].replace("bpm", "")
    return f"edm drums bpm {bpm}"

In [8]:
def create_musicgen_csv(data_directory_path, output_csv):
    with open(output_csv, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerow(["audio", "caption"])
        
        for filename in os.listdir(data_directory_path):
            if (os.path.isfile(os.path.join(data_directory_path, filename)) and 
                (filename.endswith(".wav")) and 
                (not filename.startswith("."))):
                filepath = os.path.abspath(os.path.join(data_directory_path, filename))
                caption = generate_caption(filename)
                writer.writerow([filepath, caption])

In [9]:
create_musicgen_csv("./edm_data", "./edm.csv")

In [10]:
dataset = DatasetDict.from_csv({"train":  "./edm.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio())

In [ ]:
!rm -rf edm_data.tar.gz #save storage

In [13]:
from datasets import load_dataset, Audio

dataset = load_dataset("csv", data_files="./edm.csv")
dataset = dataset.cast_column("audio", Audio())

dataset.save_to_disk("./audio_dataset")


Generating train split: 0 examples [00:00, ? examples/s]

Saving the dataset (0/15 shards):   0%|          | 0/3780 [00:00<?, ? examples/s]

In [14]:
!rm -rf ./musicgen-edm-output

In [ ]:

!python ./musicgen-dreamboothing/dreambooth_musicgen.py \
--use_lora \
--model_name_or_path "facebook/musicgen-small"\
--dataset_name  "./audio_dataset"\
--pad_token_id 0 \
--decoder_start_token_id 0\
--text_column_name caption \
--target_audio_column_name audio \
--train_split_name train \
--do_train \
--do_eval False \
--output_dir ./musicgen-edm-output \
--per_device_train_batch_size 1 \
--gradient_accumulation_steps 4 \
--learning_rate 5e-5 \
--num_train_epochs 15 \
--logging_steps 10 \
--save_steps 500 \
--save_total_limit 2 \
--generation_max_length 128 \
--report_to none


2025-05-31 18:04:32.502421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748714672.525004     195 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748714672.531979     195 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Generating train split: 3780 examples [00:16, 224.37 examples/s]
config.json: 100%|█████████████████████████| 7.87k/7.87k [00:00<00:00, 23.8MB/s]
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--musicgen-small/snapshots/4c8334b02c6ec4e8664a91979669a501ec497792/config.json
Model config MusicgenConfig {
  "architectures": [
    "MusicgenForConditionalGeneration"
  ],
  "audio_encoder":

In [ ]:
# Load processor and model

processor = AutoProcessor.from_pretrained("./musicgen-edm-output")
model = AutoModelForTextToWaveform.from_pretrained(
    "./musicgen-edm-output",
    torch_dtype=torch.bfloat16 #you might need to change this: torch.float16 for V100; torch.bfloat16 for A100
).to(device)

# Generate from prompt
inputs = processor(text="160 bpm", return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(**inputs, do_sample=True, max_new_tokens= 700,guidance_scale=1)

# Convert and save
audio = output[0].cpu().float().numpy()
sr = model.config.audio_encoder.sampling_rate

sf.write("./musicgen_output.wav", audio.T, sr)

# Playback
ipd.Audio("./musicgen_output.wav")


In [20]:
!zip -r ./musicgen-edm-output.zip ./musicgen-edm-output
